In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
from collections import Counter


In [ ]:
def short_name(team):
        short_name={'Chennai Super Kings':'CSK',
    'Mumbai Indians':'MI',
    'Gujarat Titans':'GT',
    'Kolkata Knight Riders':'KKR',
    'Punjab Kings':'PBKS',
    'Sunrisers Hyderabad':'SRH',
    'Rajasthan Royals':'RR',
    'Lucknow Super Giants':'LSG',
    'Delhi Capitals':'DC',
    'Royal Challengers Bangalore':'RCB'
    }
        return short_name[team]

In [ ]:
def get_moth(url):
    cricbuzz_highlights_url=url
    cricbuzz_highlights_url=cricbuzz_highlights_url.replace('cricket-full-commentary','cricket-scores')
    cricbuzz_highlights_url
    page=requests.get(cricbuzz_highlights_url)
    soup=BeautifulSoup(page.content)
    try: 
        win=soup.find('div',{'class':'cb-col cb-col-100 cb-min-stts cb-text-complete'}).text
        #cb-col cb-col-100 cb-mini-col cb-min-comp ng-scope
        #cb-link-undrln ng-binding
        a=soup.find('span',{'class':'cb-text-gray cb-font-12'})
        try:
            man_of_the_match=a.next_sibling.next_sibling.next_sibling.text
        except:
            man_of_the_match=None
    except:
        win = None
        man_of_the_match=None
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['win']=win
    df['man_of_the_match']=man_of_the_match
    df.to_csv(f'files/2021/{file_name}.csv',index=False)

In [ ]:
def find_toss(url):
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss=df['toss'][0]
        toss_win=toss[:toss.find('have')]
        toss_choose=toss.split(' ')[-1]

        df['Toss_winner']=short_name(toss_win.strip())
        df['toss_chosen']=toss_choose
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')
        pass
#find_toss('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [ ]:

def find_innings(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
       
    try :
        toss_winner=df['Toss_winner'][0]
        #print(toss_winner)
        toss_choose=df['toss_chosen'][0]
        #print(toss_choose)
        if toss_choose=='field':
            #print('fielding')
           
            #print(df[df['innings']== f'{toss_winner} Inns'.strip()]['innings'])
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            
            #print(df)
        else:
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            
            
           
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')

#find_innings('https://www.cricbuzz.com/cricket-full-commentary/66442/csk-vs-kkr-61st-match-indian-premier-league-2023')

In [ ]:
def find_bowling_team(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss_winner=df['Toss_winner'][0]
        toss_choose=df['toss_chosen'][0]
        a_team=short_name(df['team_a'][0])
        b_team=short_name(df['team_b'][0])
        teams=[a_team,b_team]
        for i in teams:
            if i==toss_winner:
                teams.remove(i)
        opt=teams[0]
        if toss_choose=='bat':
            df.loc[df['innings']=='1st Inns','batting_team']=toss_winner
            df.loc[df['innings']=='1st Inns','bowling_team']=opt
            df.loc[df['innings']=='2nd Inns','batting_team']=opt
            df.loc[df['innings']=='2nd Inns','bowling_team']=toss_winner
        if toss_choose=='field':
            df.loc[df['innings']=='1st Inns','batting_team']=opt
            df.loc[df['innings']=='1st Inns','bowling_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','batting_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','bowling_team']=opt
            
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except Exception as e:
        print(e)
       
#find_bowling_team('https://www.cricbuzz.com/cricket-full-commentary/66519/kkr-vs-gt-39th-match-indian-premier-league-2023')

In [ ]:
def find_batsman_bowler(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    df['Batsman']=''
    df['Bowler']=''
    #display(df)
    for i,row in enumerate(df['Commentary Text'].values):
        text=row.split(',')[0]
        text=text.split(' to ')
        batsman=text[1]
        bowler=text[0]
        df.loc[i,['Batsman']]=batsman
        #df['Batsman'][i]=batsman
        df.loc[i,['Bowler']]=bowler

        #df['Bowler'][i]=bowler
    #display(df)
    df.to_csv(f'files/2023/{file_name}.csv',index=False)
        

#find_batsman_bowler('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [ ]:
texts=[]
#find the runs by ball 
def find_runs(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['runs']=''
    df['extra']=''
    for i,comment in enumerate(df['Commentary Text'].values):
        text=comment.split(',')[1].strip().lower()
        #print('text=',text)
        texts.append(text)
        if text.startswith('1 run'.casefold()):
            df.loc[i,['runs']]= 1
            df.loc[i,['extra']]= 0
        elif text.startswith('2 runs'.casefold()):
            text= 2
            df.loc[i,['runs']]= 2
            df.loc[i,['extra']]= 0
        elif text.startswith('no run'.casefold()):
            df.loc[i,['runs']]= 0
            df.loc[i,['extra']]= 0
        elif text.startswith('3 runs'.casefold()):
            df.loc[i,['runs']]= 3
            df.loc[i,['extra']]= 0
        elif text.startswith('FOUR'.casefold()):
            df.loc[i,['runs']]= 4
            df.loc[i,['extra']]= 0
        elif text.startswith('SIX'.casefold()):
            df.loc[i,['runs']]= 6
            df.loc[i,['extra']]= 0
        elif text.startswith(('byes'.casefold(),'leg'.casefold())):
            if str(comment.split(',')[2]).lstrip().lower()=='four':
                df.loc[i,['extra']]= 4
            elif str(comment.split(',')[2]).lstrip().lower()=='six':
                df.loc[i,['extra']]= 6
            elif str(comment.split(',')[2]).lstrip().lower()=='1 run':
                df.loc[i,['extra']]= 1
            elif str(comment.split(',')[2]).lstrip().lower()=='2 runs':
                df.loc[i,['extra']]= 2
            elif str(comment.split(',')[2]).lstrip().lower()=='3 runs':
                df.loc[i,['extra']]= 3
            else:
                df.loc[i,['extra']]=0
            df.loc[i,['runs']]= 'byes'
        elif text.startswith('no ball'.casefold()):
            if str(comment.split(',')[2]).lstrip().lower()=='four':
                df.loc[i,['extra']]= '1+4'
            elif str(comment.split(',')[2]).lstrip().lower()=='six':
                df.loc[i,['extra']]='1+6'
            elif str(comment.split(',')[2]).lstrip().lower()=='1 run':
                df.loc[i,['extra']]= 1
            elif str(comment.split(',')[2]).lstrip().lower()=='2 runs':
                df.loc[i,['extra']]= 2
            elif str(comment.split(',')[2]).lstrip().lower()=='3 runs':
                df.loc[i,['extra']]= 3
            else:
                df.loc[i,['extra']]=1
            df.loc[i,['runs']]= 'no ball'
        elif text.startswith('out'.casefold()):
            df.loc[i,['runs']]= 'out'
            df.loc[i,['extra']]= 'out'
        elif text.startswith('wide'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 1
        elif text.startswith('2 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 2
        elif text.startswith('3 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 3
        elif text.startswith('4 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 4
        elif text.startswith('5 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 5
        
        #print(text)
    #display(df.sample(5))
    df.to_csv(f'files/2021/{file_name}.csv',index=False)

#find_runs('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [ ]:
# find the length of the each ball
def find_length(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    length=[]
    none=[]
    df=pd.read_csv(f'files/2023/{file_name}.csv')

    #display(df)
    for i,line in enumerate(df["Commentary Text"]):
        match=re.search('short|slot|full-toss|back of|good|tosse|full|on a length|yorker|of a length|length delivery|length ball|length on|length,',line,re.IGNORECASE)

        length_balls=['on a length','of a length','length delivery','length ball','length on','length,']
    
        if match:
            
            if match.group(0).casefold()=='short':
                df.loc[i,['ball_length']]= 'Short'
            elif match.group(0).casefold()=='full':
                df.loc[i,['ball_length']]= 'Full'
            elif match.group(0).casefold()=='slot':
                df.loc[i,['ball_length']]= 'Slot'
            elif match.group(0).casefold()=='back of':
                df.loc[i,['ball_length']]= 'Back of Length'
            elif match.group(0).casefold()=='good':
                df.loc[i,['ball_length']]= 'Good'
            elif match.group(0).casefold()=='tosse' or match.group(0).casefold()=='full-toss':
                df.loc[i,['ball_length']]= 'Full toss'
            elif match.group(0).casefold()=='yorker':
                df.loc[i,['ball_length']]= 'yorker'
            elif  match.group(0).casefold() in length_balls:
                df.loc[i,['ball_length']]= 'Good'
            else:
                print(match.group(0))
                break

        else:
            df.loc[i,['ball_length']]= 'undefined'
    #display(df)
    df.to_csv(f'files/2023/{file_name}.csv',index=False)    
#find_length('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [ ]:
#used to check the commentry
def check_commentary(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    ball_line=[]
    none=[]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    print(df.info())
    for i,line in enumerate(df['Commentary Text']):
        match=re.search('down-leg|outside off|on the stumps|on the pads|on the middle|on the leg|into off|off',line,re.IGNORECASE)
        if match:
            ball_line.append(line)
    print(len(ball_line))
#check_commentary('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [ ]:
def check_commentary(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    t=[]
    l=0
    s=0
    k=0
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    
    for i,line in enumerate(df["Commentary Text"]):
        if df.loc[i,'ball_length'] =='undefined':
            s=s+1
            match=re.search('down-leg|wide|off stump|outside-off|on middle|on the stumps|the hip|into the wicket|to off|on off|leg stump',line,re.IGNORECASE)
            if match:
                #print(df.loc[i,'ball_length'])
                #print(line)
                pass
        else:
            match=re.search('down-leg|wide|outside-off|outside off|on middle|on the stumps|the hip|sticks|into the wicket|to off|on off|leg stump',line,re.IGNORECASE)
            if match:
                l=l+1
            else:
                if 'away from' in line.lower():
                    print(line)
                    k=k+1
                pass
    
                    


        
    print(k)
check_commentary('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [29]:
def find_line(url):
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    #print(df.info())
    l=0
    k=0
    s=0
    for i,line in enumerate(df["Commentary Text"]):
        if df.loc[i,'ball_length'] =='undefined':
            s=s+1
            match=re.search('down-leg|wide|at the stump|off stump|outside-off|on middle|on the stumps|the hip|sticks|into the wicket|outside leg|outside-leg|to off|on off|on leg|leg stump|leg-lined|middle-lined',line,re.IGNORECASE)
            if match:
                #print(match)
                stumps=['at the stumps','on middle','on the stumps','the hip','sticks','into the wicket','leg stump','down leg']
                off_side=['off stump','outside-off','outside off','on off','on off']
                leg_side=['outside leg','on leg','leg stump','leg-lined','outside-leg']


                if match.group(0).casefold() in stumps:
                    df.loc[i,['ball_line']]= 'stump line'
                elif match.group(0).casefold() in off_side:
                    df.loc[i,['ball_line']]= 'off side'
                elif match.group(0).casefold() in leg_side:
                    df.loc[i,['ball_line']]= 'leg side'
                elif match.group(0).casefold() == 'wide':
                    df.loc[i,['ball_line']]= 'wide' 
                else:
                    print('error',match.group(0))
        else:
            match=re.search('down-leg|wide|at the stump|off stump|outside-off|outside-leg|outside off|on middle|on the stumps|the hip|sticks|into the wicket|outside leg|to off|on off|on leg|leg stump|leg-lined|middle-lined',line,re.IGNORECASE)
            if match:
                l=l+1
                #print(match)
                stumps=['at the stumps','on middle','on the stumps','the hip','sticks','into the wicket','leg stump','down leg']
                off_side=['off stump','outside-off','outside off','on off','on off']
                leg_side=['outside leg','on leg','leg stump','leg-lined','outside-leg']


                if match.group(0).casefold() in stumps:
                    df.loc[i,['ball_line']]= 'stump line'
                elif match.group(0).casefold() in off_side:
                    df.loc[i,['ball_line']]= 'off side'
                elif match.group(0).casefold() in leg_side:
                    df.loc[i,['ball_line']]= 'leg side'
                elif match.group(0).casefold() == 'wide':
                    df.loc[i,['ball_line']]= 'wide' 
                else:
                    print('error',match.group(0))
    display(df)

find_line('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

error to off
error to off
error at the stump
error at the stump
error middle-lined


,ball,Commentary Text,innings,match_no,team_a,team_b,team_a_11,team_b_11,team_a_subs,team_b_subs,...,winner,player_of_the_match,Toss_winner,toss_chosen,Batsman,Bowler,runs,extra,ball_length,ball_line
0,19.6,"Harshal Patel to Bumrah, out Rahul Chahar Run...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Bumrah,Harshal Patel,out,out,undefined,NaN
1,19.5,"Harshal Patel to Bumrah, no run, now he almos...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Bumrah,Harshal Patel,0,0,yorker,off side
2,19.4,"Harshal Patel to Marco Jansen, out Bowled!! O...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Marco Jansen,Harshal Patel,out,out,yorker,stump line
3,19.3,"Harshal Patel to Marco Jansen, no run, oh alm...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Marco Jansen,Harshal Patel,0,0,yorker,NaN
4,19.2,"Harshal Patel to Pollard, out Caught by Washi...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Pollard,Harshal Patel,out,out,undefined,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.4,"Boult to Washington Sundar, 1 run, 135ks leng...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Washington Sundar,Boult,1,0,Good,off side
246,0.3,"Boult to Washington Sundar, no run, fuller an...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Washington Sundar,Boult,0,0,Full,wide
247,0.3,"Boult to Washington Sundar, 5 wides, full and...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Washington Sundar,Boult,wide,5,Full,wide
248,0.2,"Boult to Washington Sundar, no run, dropped! ...",1st Inns,1st,Mumbai Indians,Royal Challengers Bangalore,"Mumbai Indians (Playing XI): Rohit Sharma(c), ...",Royal Challengers Bangalore (Playing XI): Vira...,NaN,NaN,...,Royal Challengers Bangalore won by 2 wkts,Harshal Patel,RCB,field,Washington Sundar,Boult,0,0,Full,off side


In [ ]:
links=[]
with open('match_links/2023_season_match_links.txt','r') as file:
    for line in file:
        links.append(line.strip())
print(links)
# links=links[74:]
# for i in tqdm(links,colour='green'):
#     find_length(i)